In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [6]:
url = 'https://map.kakao.com'
driver = webdriver.Chrome()
driver.get(url)

# 지도 설정 이미지 클릭
img = driver.find_element(By.CLASS_NAME, 'inner_coach_layer')
img.click()

# 검색창에 '제주도 마트' 입력
search = driver.find_element(By.ID, 'search.keyword.query')
search.send_keys("제주도 마트")

driver.find_element(By.CLASS_NAME, 'go.ico_search.btn_search.active').click()

In [7]:
# 장소 더보기 클릭
btnMore = driver.find_element(By.ID, "info.search.place.more")
btnMore.click()

# 페이지 번호 수집
page = driver.find_elements(By.ID, "info.search.page > div > a")

In [ ]:
data = [] # 크롤링 데이터를 저장할 빈 리스트 생성
for i in range(1, 8):
    if i == 1:
        for j in range (1, 6):
            
            driver.find_element(By.ID, f"info.search.page.no{j}").click()
            time.sleep(2) # 2초 동안 대기
            soup = bs(driver.page_source, "lxml") # 현재 페이지의 HTML 소스를 파싱
            titles = soup.select("a.link_name")
            addresses = soup.select('.addr')
            scores = soup.select(".rating.clickArea > .score")

            title_texts = [title.text for title in titles]
            address_texts = [address.text for address in addresses]
            score_texts = [score.text for score in scores]

            # zip함수를 사용하여 세 리스트를 하나로 묶고, 딕셔너리 형태로 변환
            for title_text, address_text, score_text in zip(title_texts, address_texts, score_texts):
                data.append({'이름': title_text, '주소': address_text, '평점': score_text})
    else:
        more = driver.find_element(By.ID, "info.search.page.next")
        more.click()
        for j in range (1, 6):
            driver.find_element(By.ID, f"info.search.page.no{j}").click()
            time.sleep(2) 
            soup = bs(driver.page_source, "lxml")
            titles = soup.select("a.link_name")
            addresses = soup.select('.addr')
            scores = soup.select(".rating.clickArea > .score")

            title_texts = [title.text for title in titles]
            address_texts = [address.text for address in addresses]
            score_texts = [score.text for score in scores]
            for title_text, address_text, score_text in zip(title_texts, address_texts, score_texts):
                data.append({'이름': title_text, '주소': address_text, '평점': score_text})

In [5]:
len(data) # 리스트의 원소 개수

500

In [6]:
df = pd.DataFrame(data) # 데이터프레임으로 만들기
df.to_csv("제주도 마트_크롤링 결과.csv")

In [7]:
df.tail()

,이름,주소,평점
495,정희마트,\n제주특별자치도 제주시 동문로21길 21\n(지번) 건입동 1043-1\n,\n별점\n0.0\n\n\n\n0건\n
496,한아름 낚시마트,\n제주특별자치도 서귀포시 남원읍 태위로 932\n(지번) 남원읍 태흥리 483-2\n,\n별점\n0.0\n\n\n\n0건\n
497,유신종합마트,\n제주특별자치도 제주시 동문로2길 9\n(지번) 일도일동 1481-2\n,\n별점\n0.0\n\n\n\n0건\n
498,조천농협하나로마트 주차장,\n제주특별자치도 제주시 조천읍 신북로 240\n(지번) 조천읍 조천리 2325-1\n,\n별점\n0.0\n\n\n\n0건\n
499,코사마트 추자점,\n제주특별자치도 제주시 추자면 추자로 18-1\n(지번) 추자면 대서리 8-9\n,\n별점\n5.0\n\n\n\n1건\n
